In [ ]:
project = ''
bucket = ''
sample_file = bucket + 'sample-web-log.csv'
bq_destination_table = 'dublin.datalab_web_logs' #format: dataset.table

### Check out the GCS help contents

In [ ]:
%%gcs --help

### Read contents of our sample file into a variable

In [ ]:
%%gcs read --object $sample_file --variable web_logs

### Let's have a look at the file

In [ ]:
print(web_logs[0:500])

### Load the contents into a dataframe

In [ ]:
import pandas as pd
from io import BytesIO

df = pd.read_csv(BytesIO(web_logs))

### And take a look

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.dtypes

### Fix the event_time: 'Error: Timed out', and extract product ids from: url

In [ ]:
df = df[df.event_time != 'Error: Timed out']
df.event_time = pd.to_datetime(df.event_time)

df = df.rename(columns={'url': 'pid'})
df.pid = df.pid.str.extract('pid=(?P<pid>\d*-\d*)', expand=False)

df.head()

### Write the data to BigQuery

In [ ]:
df.to_gbq(bq_destination_table, project, if_exists='replace')

### Checkout the BigQuery help contents

In [ ]:
%%bq --help

### Describe the table

In [ ]:
%%bq tables describe --name bigquery-public-data.census_bureau_international.midyear_population

### Sample the data

In [ ]:
%%bq sample --table bigquery-public-data.census_bureau_international.midyear_population --count 5

In [ ]:
%%bq query --name data
SELECT *
FROM `bigquery-public-data.census_bureau_international.midyear_population`
WHERE year = 2017

### And charting

In [ ]:
%%chart --help

In [ ]:
%%chart geo --data data --fields country_code,midyear_population